# WB4.3 Introducción a SQL

En esta cuaderno de trabajo se explorarán la base de datos `music.db` mediante consultas de SQL. La base de datos contiene tres tablas:
* `songs`, que contiene un identificador `_id`, el título de la canción `title` y el album `album`. 
* `albums`, que contiene un identificador `_id`, el nombre del album `name` y el artista `artist`. 
* `artists`, que contiene un identificador `_id` y su nombre `name`. 

Completa las siguientes instrucciones de acuerdo a lo visto en el tema.

## 1. Acceso a los Datos con Queries.

1. Crea la conexión y un cursor a la base de datos.

In [1]:
import sqlite3
import os

DB_PATH = "music.db"

if not os.path.exists(DB_PATH):
    raise FileNotFoundError(f"No se encontró '{DB_PATH}' en el directorio actual: {os.getcwd()}")

conn = sqlite3.connect(DB_PATH)

conn.row_factory = sqlite3.Row

cur = conn.cursor()

2. Explora todas las columnas en todas antes mencionadas tablas. Muestra los primeros 10 registros.

In [2]:
tablas = ["songs", "albums", "artists"]

for tabla in tablas:
    cur.execute(f"PRAGMA table_info({tabla});")
    columnas = [col[1] for col in cur.fetchall()]
    print("Columnas:", columnas)
    
    cur.execute(f"SELECT * FROM {tabla} LIMIT 10;")
    registros = cur.fetchall()
    for fila in registros:
        print(dict(zip(columnas, fila)))

Columnas: ['_id', 'track', 'title', 'album']
{'_id': 1, 'track': 2, 'title': "I Can't Quit You Baby", 'album': 343}
{'_id': 2, 'track': 1, 'title': 'Taking the Easy Way Out Again', 'album': 311}
{'_id': 3, 'track': 6, 'title': "Let's Have A Party", 'album': 260}
{'_id': 4, 'track': 7, 'title': 'Flaming Telepaths', 'album': 104}
{'_id': 5, 'track': 11, 'title': "Yearnin'", 'album': 37}
{'_id': 6, 'track': 1, 'title': 'Bat Out Of Hell', 'album': 290}
{'_id': 7, 'track': 5, 'title': 'Evil Woman', 'album': 143}
{'_id': 8, 'track': 7, 'title': 'Rat Salad', 'album': 50}
{'_id': 9, 'track': 8, 'title': 'Young is a World', 'album': 402}
{'_id': 10, 'track': 3, 'title': 'Sam With The Showing Scalp Flat Top', 'album': 429}
Columnas: ['_id', 'name', 'artist']
{'_id': 1, 'name': 'Tales of the Crown', 'artist': 16}
{'_id': 2, 'name': 'The Masquerade Ball', 'artist': 16}
{'_id': 3, 'name': 'Grace', 'artist': 159}
{'_id': 4, 'name': 'Behind Closed Doors', 'artist': 147}
{'_id': 5, 'name': 'Day & Age'

3. Iron Maiden tiene como `_id = 8`en la tabla de artistas. Emplea este id para identificar sus álbumes en la tabla `albums`. Observa que la columna `artist` hace referencia a los id de los artistas. 

In [3]:
artist_id = 8

cur.execute("""
    SELECT _id, name 
    FROM albums
    WHERE artist = ?
""", (artist_id,))

albums_iron_maiden = cur.fetchall()

print("Álbumes de Iron Maiden:")
for album in albums_iron_maiden:
    print(dict(album))

Álbumes de Iron Maiden:
{'_id': 236, 'name': 'The Number of the Beast'}
{'_id': 412, 'name': 'Powerslave'}
{'_id': 420, 'name': 'Seventh Son Of A Seventh Son'}


4. Muestras las canciones del album "The Number of the Beast". Presenta tus resultados en orden alfabético.

In [4]:
album_name = "The Number of the Beast"

cur.execute("""
    SELECT s.title
    FROM songs s
    JOIN albums a ON s.album = a._id
    WHERE a.name = ?
    ORDER BY s.title ASC
""", (album_name,))

canciones = cur.fetchall()

print(f"Canciones del álbum '{album_name}':")
for cancion in canciones:
    print(cancion['title'])

Canciones del álbum 'The Number of the Beast':
22 Acacia Avenue (1998 Digital Remaster)
Children Of The Damned (1998 Digital Remaster)
Gangland (1998 Digital Remaster)
Hallowed Be Thy Name (1998 Digital Remaster)
Invaders (1998 Digital Remaster)
Run To The Hills (1998 Digital Remaster)
The Number Of The Beast (1998 Digital Remaster)
The Prisoner (1998 Digital Remaster)
Total Eclipse (1998 Digital Remaster)


## 2. Agregaciones y Agrupamientos de Datos.

1. ¿Cuantas canciones, albumes y artistas se tienen registrados en la base de datos?

In [5]:
tablas = ["songs", "albums", "artists"]

for tabla in tablas:
    cur.execute(f"SELECT COUNT(*) AS total FROM {tabla}")
    total = cur.fetchone()["total"]
    print(f"Total de registros en {tabla}: {total}")

Total de registros en songs: 5350
Total de registros en albums: 439
Total de registros en artists: 201


2. Agrupa la tabla albums por artista para identificar qué cuales son los 5 artistas que tienen más albumes.

In [6]:
cur.execute("""
    SELECT ar.name AS artist_name, COUNT(al._id) AS album_count
    FROM albums al
    JOIN artists ar ON al.artist = ar._id
    GROUP BY ar._id
    ORDER BY album_count DESC
    LIMIT 5
""")

top_artistas = cur.fetchall()

3. Presenta el nombre de los artistas identificados en el punto anterior.

In [7]:
nombres_artistas = [artista['artist_name'] for artista in top_artistas]

print("Artistas con más álbumes:")
for nombre in nombres_artistas:
    print(nombre)

Artistas con más álbumes:
Black Sabbath
Axel Rudi Pell
Led Zeppelin
Deep Purple
Aerosmith


4. ¿Cuál es el album con más canciones? Investiga sobre este album en la red.

In [8]:
cur.execute("""
    SELECT al.name AS album_name, COUNT(s._id) AS song_count
    FROM songs s
    JOIN albums al ON s.album = al._id
    GROUP BY al._id
    ORDER BY song_count DESC
    LIMIT 1
""")

album_mas_canciones = cur.fetchone()

print(f"Álbum con más canciones: {album_mas_canciones['album_name']} ({album_mas_canciones['song_count']} canciones)")

Álbum con más canciones: Cornology (72 canciones)


Cornology salió en 1992, es una compilación de todas las canciones de la banda The Bonzo Dog Band, dividiendose entre "The Intro", "The Outro" y "Dog Ends".

## 3. Combinación de Tablas.

1. Muestra todas el nombre canciones del album "Seventh Son Of A Seventh Son" que tiene por '_id = 420'. Presenta la lista en el *track order* (`track`).

In [9]:
album_id = 420

cur.execute("""
    SELECT title, track
    FROM songs
    WHERE album = ?
    ORDER BY track ASC
""", (album_id,))

canciones = cur.fetchall()

print(f"Canciones del álbum Seventh Son Of A Seventh Son:")
for cancion in canciones:
    print(f"Track {cancion['track']}: {cancion['title']}")

Canciones del álbum Seventh Son Of A Seventh Son:
Track 1: Moonchild (1998 Digital Remaster)
Track 2: Infinite Dreams (1998 Digital Remaster)
Track 3: Can I Play With Madness (1998 Digital Remaster)
Track 4: The Evil That Men Do (1998 Digital Remaster)
Track 5: Seventh Son Of A Seventh Son (1998 Digital Remaster)
Track 6: The Prophecy (1998 Digital Remaster)
Track 7: The Clairvoyant (1998 Digital Remaster)
Track 8: Only The Good Die Young (1998 Digital Remaster)


2. Obtén todas las canciones de Aeroesmith en orden alfabético. Solo incluye el título en la salida ¿Cuántas son?

In [10]:
artist_name = "Aerosmith"

cur.execute("""
    SELECT s.title
    FROM songs s
    JOIN albums a ON s.album = a._id
    JOIN artists ar ON a.artist = ar._id
    WHERE ar.name = ?
    ORDER BY s.title ASC
""", (artist_name,))

canciones_aerosmith = cur.fetchall()

for cancion in canciones_aerosmith:
    print(cancion['title'])

(Dulcimer Stomp) The Other Side
(Going Down) Love In An Elevator
(Hoodoo) Voodoo Medicine Man
(Water Song) Janie's Got A Gun
Adam's Apple
Adam's Apple
Ain't That A Bitch
Amazing
Amazing
Angel
Angel
Attitude Adjustment
Avant Garden
Back In The Saddle
Beyond Beautiful
Big Ten Inch Record
Blind Man
Bone To Bone (Coney Island White Fish Boy)
Boogie Man
Can't Stop Messin'
Cheese Cake
Chip Away The Stone
Chip Away The Stone
Chiquita
Come Together
Crash
Crazy
Crazy
Critical Mass
Cryin'
Cryin'
Darkness
Deuces Are Wild
Don't Get Mad Get Even
Dream On
Dream On
Drop Dead Gorgeous
Dude (Looks Like A Lady)
Dude (Looks Like A Lady)
Dude (Looks Like A Lady)- live
Eat The Rich
Eat The Rich
F.I.N.E.
Face
Fallen Angels
Falling In Love (Is Hard On The Knees)
Falling Off
Fever
Flesh
Fly Away From Here
Full Circle
Get A Grip
Girl Keeps Coming Apart
Gotta Love It
Gypsy Boots
Hangman Jury
Heart's Done Time
Hole In My Soul
I Ain't Got You
I'm Down
Intro
Jaded
Jailbait
Janie's Got A Gun
Just Push Play
Kiss You

In [11]:
print(f"\nTotal de canciones: {len(canciones_aerosmith)}")


Total de canciones: 151


## Referencias
* Actividad basada en SQLite-Exercises: https://github.com/IancuIulian/SQLite-Exercises